<a href="https://colab.research.google.com/github/darkraithromb/next-platform-starter/blob/main/Mpox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from keras.applications import Xception, DenseNet169
from keras.layers import Concatenate, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set the seed for reproducibility
np.random.seed(42)

# Define the path to the dataset
train_dir = 'path_to_your_train_directory'
test_dir = 'path_to_your_test_directory'

# Define the image dimensions
img_height, img_width = 150, 150

# Define the batch size
batch_size = 16

# Define the number of epochs
epochs = 10

# Define the learning rate
learning_rate = 0.0001

# Define the early stopping patience
patience = 5

# Define the reduce learning rate patience
reduce_lr_patience = 3

# Define the data augmentation parameters
data_augmentation = {
    'rotation_range': 50,
    'width_shift_range': 0.2,
    'height_shift_range': 0.2,
    'shear_range': 0.25,
    'zoom_range': 0.1,
    'channel_shift_range': 20,
    'horizontal_flip': True,
    'vertical_flip': True,
    'rescale': 1/255
}

# Create the data generators for training and testing
train_datagen = ImageDataGenerator(**data_augmentation)
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained Xception model
xception_base = Xception(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Load the pre-trained DenseNet-169 model
densenet_base = DenseNet169(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base layers
for layer in xception_base.layers:
    layer.trainable = False

for layer in densenet_base.layers:
    layer.trainable = False

# Define the ensemble model
x = xception_base.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)

y = densenet_base.output
y = GlobalAveragePooling2D()(y)
y = Dense(128, activation='relu')(y)

z = Concatenate()([x, y])
z = Dense(4, activation='softmax')(z)

model = Model(inputs=[xception_base.input, densenet_base.input], outputs=z)

# Compile the model
model.compile(optimizer=Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

# Define the early stopping and reduce learning rate callbacks
early_stopping = EarlyStopping(patience=patience, min_delta=0.001)
reduce_lr = ReduceLROnPlateau(patience=reduce_lr_patience, min_lr=0.00001)

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Plot the training and validation accuracy and loss
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Loss: {test_loss:.3f}')
print(f'Test Accuracy: {test_acc:.3f}')

# Get the predictions on the test set
predictions = model.predict(test_generator)

# Convert the predictions to class labels
predicted_classes = np.argmax(predictions, axis=1